## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-19-10-49-22 +0000,bbc,Who are the released hostages?,https://www.bbc.com/news/articles/cpvl9k4mw8no...
1,2025-10-19-10-49-00 +0000,wsj,Seven-Minute Heist at Louvre Leaves Museum Mis...,https://www.wsj.com/world/europe/seven-minute-...
2,2025-10-19-10-38-32 +0000,bbc,Israel launches air strikes in Gaza accusing H...,https://www.bbc.com/news/articles/cm2elr16jjqo...
3,2025-10-19-10-34-46 +0000,nyt,Louvre Closed After Robbery,https://www.nytimes.com/2025/10/19/world/europ...
4,2025-10-19-10-34-29 +0000,bbc,"Government vows to create 400,000 jobs in ener...",https://www.bbc.com/news/articles/c3vnr45x5qyo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2385/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
74,trump,28
64,no,17
65,kings,15
11,israel,11
15,gaza,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
41,2025-10-19-06-45-06 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...,86
58,2025-10-19-03-33-45 +0000,missionlocal,"Photos: Defiance, solidarity fill S.F. streets...",https://missionlocal.org/2025/10/photos-defian...,85
120,2025-10-18-18-45-48 +0000,cbc,'Trump must go': Large crowds join 'No Kings' ...,https://www.cbc.ca/news/world/no-kings-protest...,83
44,2025-10-19-06-11-20 +0000,bbc,Millions join anti-Trump 'No Kings' protests a...,https://www.bbc.com/news/articles/c93xgyp1zv4o...,82
73,2025-10-19-00-07-12 +0000,nyt,Scenes from the ‘No Kings’ Anti-Trump Protests...,https://www.nytimes.com/2025/10/18/us/politics...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
41,86,2025-10-19-06-45-06 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...
2,44,2025-10-19-10-38-32 +0000,bbc,Israel launches air strikes in Gaza accusing H...,https://www.bbc.com/news/articles/cm2elr16jjqo...
94,35,2025-10-18-22-04-02 +0000,nypost,NYC police union leader calls out Zohran Mamda...,https://nypost.com/2025/10/18/us-news/nyc-dete...
98,33,2025-10-18-21-08-36 +0000,nypost,Jeffrey Epstein sent nasty emails to billionai...,https://nypost.com/2025/10/18/us-news/jeffrey-...
107,32,2025-10-18-20-09-11 +0000,nypost,Zelensky’s drone attacks on oil refineries cau...,https://nypost.com/2025/10/18/world-news/zelen...
108,27,2025-10-18-19-51-55 +0000,nypost,Parents of Long Island teacher accused of sext...,https://nypost.com/2025/10/18/us-news/parents-...
50,25,2025-10-19-04-31-22 +0000,nyt,Sanae Takaichi Is Set to Be Japan’s First Fema...,https://www.nytimes.com/2025/10/19/world/asia/...
109,25,2025-10-18-19-48-40 +0000,cbc,U.S. sending survivors of strike on alleged 'd...,https://www.cbc.ca/news/world/us-boat-strike-c...
118,24,2025-10-18-19-04-19 +0000,nypost,Obama-era Iran nuclear non-proliferation deal ...,https://nypost.com/2025/10/18/world-news/obama...
67,24,2025-10-19-01-19-22 +0000,nypost,Twisted NY man who bragged he was planning to ...,https://nypost.com/2025/10/18/us-news/david-co...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
